In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import winsound
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report

from keras.utils import np_utils

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy


print(f" Found and Using {len(tf.config.experimental.list_physical_devices('GPU'))} GPU")

 Found and Using 1 GPU


In [2]:
def scale_x_encode_y(df, target_col):

    x = df.loc[:, df.columns != target_col]
    y = df[target_col]

    x_scaled = StandardScaler().fit_transform(x.values)
    
    
    encoder = LabelEncoder()
    encoder.fit(y)
    y_encoded = encoder.transform(y)
    y_encoded = np_utils.to_categorical(y_encoded)
    
    y_map = dict(zip(encoder.transform(encoder.classes_),encoder.classes_))
 
    return(x_scaled, y_encoded, y_map)

In [3]:
df = pd.read_csv('data/IRIS.csv')

print(len(df))
df.head(10)

150


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [4]:
x_scaled, y_encoded, y_map = scale_x_encode_y(df, 'species')

In [5]:
train_x, valid_x, train_y, valid_y = train_test_split(x_scaled, y_encoded,train_size = 0.8,random_state = 1)

In [6]:
model = Sequential([
    Dense(units = 16, activation='relu', input_shape=[len(train_x[0])]),
    Dense(units = 32, activation='relu'),
    Dense(units = 3, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),    
    metrics=['accuracy'])

In [8]:
epoch_val         = 30
batch_size_val    = 10

verbose_val       = 2
workers_val       = -1

In [9]:
model.fit(x=x_scaled, 
          y=y_encoded, 
          epochs=epoch_val,
          batch_size = batch_size_val,
          validation_data = (valid_x, valid_y),
          workers= workers_val,
         verbose=verbose_val)

Epoch 1/30
15/15 - 0s - loss: 1.0629 - accuracy: 0.5333 - val_loss: 0.9452 - val_accuracy: 0.7000
Epoch 2/30
15/15 - 0s - loss: 0.9335 - accuracy: 0.5467 - val_loss: 0.8454 - val_accuracy: 0.7667
Epoch 3/30
15/15 - 0s - loss: 0.8340 - accuracy: 0.5733 - val_loss: 0.7563 - val_accuracy: 0.7667
Epoch 4/30
15/15 - 0s - loss: 0.7412 - accuracy: 0.6067 - val_loss: 0.6790 - val_accuracy: 0.6667
Epoch 5/30
15/15 - 0s - loss: 0.6638 - accuracy: 0.7267 - val_loss: 0.6114 - val_accuracy: 0.7000
Epoch 6/30
15/15 - 0s - loss: 0.5924 - accuracy: 0.8133 - val_loss: 0.5564 - val_accuracy: 0.7000
Epoch 7/30
15/15 - 0s - loss: 0.5327 - accuracy: 0.8200 - val_loss: 0.5088 - val_accuracy: 0.7333
Epoch 8/30
15/15 - 0s - loss: 0.4828 - accuracy: 0.8267 - val_loss: 0.4720 - val_accuracy: 0.7667
Epoch 9/30
15/15 - 0s - loss: 0.4418 - accuracy: 0.8267 - val_loss: 0.4411 - val_accuracy: 0.7667
Epoch 10/30
15/15 - 0s - loss: 0.4086 - accuracy: 0.8267 - val_loss: 0.4191 - val_accuracy: 0.7667
Epoch 11/30
15/15 -

In [10]:
predictions = model.predict(valid_x)
rounded_predictions = np.argmax(predictions, axis = -1)
rounded_valid_y = np.argmax(valid_y, axis = -1)

print(classification_report(rounded_valid_y, rounded_predictions, target_names = y_map.values()))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       1.00      0.85      0.92        13
 Iris-virginica       0.75      1.00      0.86         6

       accuracy                           0.93        30
      macro avg       0.92      0.95      0.92        30
   weighted avg       0.95      0.93      0.94        30



# Load and Test model

model.save('data/model/iris_model.h5')
model_new = load_model('data/model/iris_model.h5')

predictions = model_new.predict(valid_x)
rounded_predictions = np.argmax(predictions, axis = -1)
rounded_valid_y = np.argmax(valid_y, axis = -1)

print(classification_report(rounded_valid_y, rounded_predictions, target_names = y_map.values()))